In [ ]:
import os
import sys
sys.path.append('../../lib')
import SDS
import datetime, pytz
import glob
import obspy
inputdirroot =  '/data/SEISAN_DB/WAV/DSNC_'
outputdir = '../../data/continuous/SDS'
dbout = f"../../db/dbMontserrat"
print(os.getcwd())
sdsobj = SDS.SDSobj(outputdir)

def sds2db(SDS_TOP, jday):
    allfiles = glob.glob(os.path.join(SDS_TOP, '*', '*', '*', '*.D', f'*{jday}'))
    allfilesstr = " ".join(allfiles)
    os.system(f"miniseed2db {allfilesstr} {dboutday}")

utc = pytz.timezone('UTC')
start_dt_utc = utc.localize(datetime.datetime(2001, 2, 25, 0, 0, 0))
end_dt_utc = utc.localize(datetime.datetime(2001, 3, 5, 0, 0, 0))
delta = datetime.timedelta(days=1)
mseeddir = 'seisan2mseed'
if not os.path.isdir(mseeddir):
    os.makedirs(mseeddir)

os.system(f"rm -rf {outputdir}/* {dbout}.*")
if not os.path.isdir(outputdir):
    os.makedirs(outputdir)
laststarttime=0
while (start_dt_utc <= end_dt_utc):
    print(start_dt_utc, end="\n")
    ymd = start_dt_utc.strftime("%Y%m%d")
    chuckfile = f"chuckmseedblocks{ymd}.msd"
    startepoch = int(start_dt_utc.timestamp())
    endepoch = startepoch + 86400
    jday = start_dt_utc.strftime("%j")
    
    yyyy = start_dt_utc.strftime("%Y")
    mm = start_dt_utc.strftime("%m")
    dd = start_dt_utc.strftime("%d")
    inputdir = f"{inputdirroot}/{yyyy}/{mm}"
    dboutday = f"{dbout}{ymd}"

    pstart_dt_utc = start_dt_utc - delta
    pyyyy = pstart_dt_utc.strftime("%Y")
    pmm = pstart_dt_utc.strftime("%m")
    pdd = pstart_dt_utc.strftime("%d")
    pinputdir = f"{inputdirroot}/{pyyyy}/{pmm}"
    
    allfiles0 = glob.glob(os.path.join(pinputdir, f'{pyyyy}-{pmm}-{pdd}-23[45]*S.MVO___*'))
    allfiles1 = glob.glob(os.path.join(inputdir, f'{yyyy}-{mm}-{dd}*S.MVO___*'))
    allfiles = sorted(allfiles0 + allfiles1)
    
    print('- Found %d files' % len(allfiles))
    for file in allfiles:
        print(f'- Processing {file}')
        st = obspy.core.read(file, format='seisan')
        thisstarttime = st[0].stats.starttime
        if thisstarttime - laststarttime == 1201.0: # should be 20 * 60 s = 1200 s
            thisstarttime -= 1.0
        elif thisstarttime - laststarttime == 1199.0: # should be 20 * 60 s = 1200 s
            thisstarttime += 1.0
            
        for tr in st:
            tr.stats.starttime = thisstarttime
            tr.stats.sampling_rate = 75.0
            if tr.stats.channel[0:2]=='SB':
                tr.stats.channel = 'BH' + tr.stats.channel[2:]
            if tr.stats.channel[0:2]=='S ':
                tr.stats.channel = 'SH' + tr.stats.channel[2:]    
            if tr.stats.channel == 'PRS':
                tr.stats.channel = 'BDO' # microbarometer, possible an absolute, very-long-period instrument
            tr.stats.location = ""
            if tr.stats.network == "":
                tr.stats.network = 'MV'
        laststarttime = thisstarttime
        print(st)
        
        sdsobj.stream = st
        sdsobj.write()
    sds2db(outputdir, jday)
    start_dt_utc += delta

# convert original RSAM binary files to CSV files

In [ ]:
import sys
sys.path.append('../../lib')
import FLOVO
import obspy
RSAM_DIR = '../../data/continuous/RSAM'

st = obspy.core.Stream()
stime = obspy.core.UTCDateTime(1996,1,1,0,0,0)
etime = obspy.core.UTCDateTime(1996,12,31,23,59,59)
stations = ['MCPZ', 'MGAT', 'MGHZ', 'MLGT', 'MRYT', 'MSPT', 'MSSZ', 'MWHT']
rsamObj = FLOVO.readRSAMbinary(RSAM_DIR, stations, stime, etime)
rsamObj.plot()
rsamObj.write(RSAM_DIR, ext='csv')

In [3]:
import os
import sys
import glob
import obspy
from obspy.core.inventory.inventory import Inventory, read_inventory
PROJECT_DIR = os.path.join('..','..')
LIB_DIR = os.path.join(PROJECT_DIR,'src','lib')
sys.path.append(LIB_DIR)
import InventoryTools

# parse Seisan STATION0.HYP file to get station coordinates
coordinates={}
station0hypfile = '/data/SEISAN_DB/STATION0_MVO.HYP'
fptr= open(station0hypfile , 'r')
lines = fptr.readlines()
for line in lines:
    line.strip()
    if line[2:4]=='MB':
        print(line)
        station=line[2:6].strip()
        #print(line[6:8])
        lat=int(line[6:8]) + int(line[8:10])/60
        if line[10]=='.':
            lat += int(line[11:13])/3600
        else:
            lat += int(line[10:13])/36000
        if line[14]=='S':
            lat=-lat
        lon=int(line[15:17]) + int(line[17:19])/60
        if line[19]=='.':
            lon += int(line[20:22])/3600
        else:
            lon += int(line[19:22])/36000
        if line[22]=='W':
            lon=-lon      
        elev=int(line[23:27])
        #print(f"station={station} lat={lat}, lon={lon}, elev={elev}")
        coordinates[station]={'lat':lat, 'lon':lon, 'elev':elev}
fptr.close()
print(coordinates)

  MBBE1644613N 62 9609W 102

  MBBY1641861N 6212153W 161

  MBFL1644.91N 6212.75W 243

  MBFR1641.58N 6210.68W 541

  MBGA1642611N 6211317W 478

  MBGB1643942N 6213666W 253

  MBGE1641404N 6211624W 183

  MBGH1643354N 6212518W 350

  MBHA1644.39N 6210.28W 291

  MBLG1643500N 62 9735W 287

  MBLY1643.28N 6211.46W 355

  MBMH1646.58N 6211.55W 270

  MBRV1648.68N 6211.76W 231

  MBRY1642234N 62 9190W 355

  MBSS1641225N 62 9616W 395

  MBWH1644531N 6211456W 407

{'MBBE': {'lat': 16.75036111111111, 'lon': -62.166916666666665, 'elev': 102}, 'MBBY': {'lat': 16.707250000000002, 'lon': -62.20425, 'elev': 161}, 'MBFL': {'lat': 16.75861111111111, 'lon': -62.22083333333334, 'elev': 243}, 'MBFR': {'lat': 16.699444444444445, 'lon': -62.18555555555555, 'elev': 541}, 'MBGA': {'lat': 16.71697222222222, 'lon': -62.192138888888884, 'elev': 478}, 'MBGB': {'lat': 16.74283333333333, 'lon': -62.23516666666667, 'elev': 253}, 'MBGE': {'lat': 16.694555555555556, 'lon': -62.20066666666666, 'elev': 183}, 'MBGH':

In [4]:
# read stationXML for each trace
stationXMLsrcDIR = '/data/SEISAN_DB/CAL'
stationXMLfiles = sorted(glob.glob(os.path.join(stationXMLsrcDIR, 'station.MV.*..[BSEH]*.xml')))
invAll = None
for xmlfile in stationXMLfiles:
    print(xmlfile)
    this_inv = read_inventory(xmlfile)
    #print(this_inv)
    trace_ids = InventoryTools.inventory2traceid(this_inv, force_location_code='')
    print(trace_ids)
    for seed_id in trace_ids:
        #print(this_inv.get_channel_metadata(id))
        net,sta,chan,loc=seed_id.split('.')
        station_coords = coordinates[sta]

        # - add coordinates
        InventoryTools.modify_inventory(this_inv, seed_id, lat=station_coords['lat'], lon=station_coords['lon'], elev=station_coords['elev'])
        #print(this_inv.get_channel_metadata(id))
    if invAll:
        # combine inventories 
        invAll.extend(this_inv)
    else:
        invAll = this_inv
        
invAll.write('../../data/response/MV.xml', format='stationxml')

/data/SEISAN_DB/CAL/station.MV.MBBE..BHE.xml
['MV.MBBE..BHE']
match found
/data/SEISAN_DB/CAL/station.MV.MBBE..BHN.xml
['MV.MBBE..BHN']
match found
/data/SEISAN_DB/CAL/station.MV.MBBE..BHZ.xml
['MV.MBBE..BHZ']
match found
/data/SEISAN_DB/CAL/station.MV.MBBY..BHE.xml
['MV.MBBY..BHE', 'MV.MBBY..HHE']
match found
match found
/data/SEISAN_DB/CAL/station.MV.MBBY..BHN.xml
['MV.MBBY..BHN', 'MV.MBBY..HHN']
match found
match found
/data/SEISAN_DB/CAL/station.MV.MBBY..BHZ.xml
['MV.MBBY..BHZ', 'MV.MBBY..HHZ']
match found
match found
/data/SEISAN_DB/CAL/station.MV.MBFL..EHE.xml
['MV.MBFL..EHE']
match found
/data/SEISAN_DB/CAL/station.MV.MBFL..EHN.xml
['MV.MBFL..EHN']
match found
/data/SEISAN_DB/CAL/station.MV.MBFL..SHZ.xml
['MV.MBFL..EHZ']
match found
/data/SEISAN_DB/CAL/station.MV.MBFR..BHE.xml
['MV.MBFR..HHE']
match found
/data/SEISAN_DB/CAL/station.MV.MBFR..BHN.xml
['MV.MBFR..HHN']
match found
/data/SEISAN_DB/CAL/station.MV.MBFR..BHZ.xml
['MV.MBFR..HHZ']
match found
/data/SEISAN_DB/CAL/station.

In [3]:
#invAll.get_channel_metadata('MV.MBBE..BHE')
trace_ids = InventoryTools.inventory2traceid(invAll, force_location_code='')

In [4]:
print(trace_ids)

['MV.MBBE..BHE', 'MV.MBBE..BHN', 'MV.MBBE..BHZ', 'MV.MBBY..BHE', 'MV.MBBY..HHE', 'MV.MBBY..BHN', 'MV.MBBY..HHN', 'MV.MBBY..BHZ', 'MV.MBBY..HHZ', 'MV.MBFL..EHE', 'MV.MBFL..EHN', 'MV.MBFL..EHZ', 'MV.MBFR..HHE', 'MV.MBFR..HHN', 'MV.MBFR..HHZ', 'MV.MBGA..BHE', 'MV.MBGA..BHN', 'MV.MBGA..BHZ', 'MV.MBGB..BHE', 'MV.MBGB..HHE', 'MV.MBGB..BHN', 'MV.MBGB..HHN', 'MV.MBGB..BHZ', 'MV.MBGB..HHZ', 'MV.MBGE..BHE', 'MV.MBGE..BHN', 'MV.MBGE..BHZ', 'MV.MBGH..BHE', 'MV.MBGH..HHE', 'MV.MBGH..HHE', 'MV.MBGH..BHN', 'MV.MBGH..HHN', 'MV.MBGH..HHN', 'MV.MBGH..BHZ', 'MV.MBGH..HHZ', 'MV.MBGH..HHZ', 'MV.MBHA..HHE', 'MV.MBHA..HHN', 'MV.MBHA..HHZ', 'MV.MBLG..HHE', 'MV.MBLG..HHN', 'MV.MBLG..HHZ', 'MV.MBLG..SHE', 'MV.MBLG..SHN', 'MV.MBLG..SHZ', 'MV.MBLY..HHE', 'MV.MBLY..HHE', 'MV.MBLY..HHN', 'MV.MBLY..HHN', 'MV.MBLY..HHZ', 'MV.MBLY..HHZ', 'MV.MBMH..SHE', 'MV.MBMH..SHN', 'MV.MBMH..SHZ', 'MV.MBRV..SHE', 'MV.MBRV..SHN', 'MV.MBRV..SHZ', 'MV.MBRY..BHE', 'MV.MBRY..HHE', 'MV.MBRY..HHE', 'MV.MBRY..BHN', 'MV.MBRY..HHN', 'MV.MBR